### Limpiar CSV de Food

### Leer CSV e Imports

In [95]:
import pandas as pd
import numpy as np

In [96]:
df = pd.read_csv('source_clean_dataset.csv')

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            255 non-null    object
 1   ingredients     255 non-null    object
 2   diet            255 non-null    object
 3   prep_time       255 non-null    int64 
 4   cook_time       255 non-null    int64 
 5   flavor_profile  255 non-null    object
 6   course          255 non-null    object
 7   state           255 non-null    object
 8   region          254 non-null    object
dtypes: int64(2), object(7)
memory usage: 18.1+ KB


### 2% de la columna dieta tenemos un unknown

In [98]:
porcentaje_unknown = 0.2
mask_unknown = np.random.rand(len(df)) < porcentaje_unknown
df.loc[mask_unknown, "diet"] = "unknown"
print(df["diet"].value_counts(dropna=False))

diet
vegetarian        189
unknown            45
non vegetarian     21
Name: count, dtype: int64


In [99]:
df_dirty = df.copy()

### Valores duplicados en columnas

In [100]:
# Porcentaje de filas duplicadas 2%
porcentaje_duplicados = 0.2

# Seleccionar filas aleatorias
df_duplicates = df_dirty.sample(frac=porcentaje_duplicados, random_state=42)

# Añadirlas al DataFrame original
df_dirty = pd.concat([df_dirty, df_duplicates], ignore_index=True)


print("Filas originales:", len(df))
print("Filas con duplicados:", len(df_dirty))


Filas originales: 255
Filas con duplicados: 306


In [101]:
df_dirty.duplicated().sum()

51

### Outlier en cook_time

In [102]:
porcentaje_outliers = 0.05
mask_outliers_cook = np.random.rand(len(df_dirty)) < porcentaje_outliers
df_dirty.loc[mask_outliers_cook, "cook_time"] = 999999  # Valor extremadamente alto como outlier

### Añadir simbolos en prep_time

In [103]:
porcentaje_simbolos = 0.03
mask_simbolos = np.random.rand(len(df_dirty)) < porcentaje_simbolos
simbolos = ["@", "#", "*", "!", "$", "%"]
df_dirty.loc[mask_simbolos, "prep_time"] = (
    df_dirty.loc[mask_simbolos, "prep_time"].astype(str) +
    np.random.choice(simbolos, size=mask_simbolos.sum())
)

### Añadir columna irrelevtante y con datos sin sentido

In [104]:
np.random.seed(42)
df_dirty["random_noise"] = np.random.choice(["foo", "bar", "baz", 123, 456], size=len(df_dirty))

### Cambiar el nombre de la columna en vez de region la llamare ingrediente_especial

In [105]:
columnas = df_dirty.columns.tolist()
region_index = columnas.index("region")
columnas[region_index] = "ingrediente_especial"  # ahora region tiene el nombre de otra columna
df_dirty.columns = columnas

### Converir a String la columna state

In [106]:
df_dirty["state"], _ = pd.factorize(df_dirty["state"])

### Mezclar formatos en la misma columna

In [107]:
df_dirty.loc[0:5, "prep_time"] = ["30", "45 min", "one hour", "60", "forty-five", "50"]

### Cambiar la palabra sweet con mayusculas en el medio o donde sea

In [108]:
porcentaje_flavor = 0.1  # 10% de los 'sweet'
mask_flavor = (df_dirty["flavor_profile"] == "sweet") & (np.random.rand(len(df_dirty)) < porcentaje_flavor)
df_dirty.loc[mask_flavor, "flavor_profile"] = np.random.choice(["SWeet", "Sweet"], size=mask_flavor.sum())

### Guardar el dataset con un coodificacion diferente

In [109]:
df_dirty.to_csv("dirty_dataset.csv", index=False, encoding="ISO-8859-1")

In [110]:
df_dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  306 non-null    object
 1   ingredients           306 non-null    object
 2   diet                  306 non-null    object
 3   prep_time             306 non-null    object
 4   cook_time             306 non-null    int64 
 5   flavor_profile        306 non-null    object
 6   course                306 non-null    object
 7   state                 306 non-null    int64 
 8   ingrediente_especial  305 non-null    object
 9   random_noise          306 non-null    object
dtypes: int64(2), object(8)
memory usage: 24.0+ KB
